### Piesi

In [ ]:
import pandas as pd
import geopy.distance
import datetime

In [ ]:
movement = pd.read_csv("points.csv")
lamps = pd.read_csv("AghLamps.csv")
movement.sort_values("time", inplace = True)

In [ ]:
def calculate_destination_coordinates(lat, lon, distance, bearing):
    destination = geopy.distance.geodesic(kilometers=distance/1000).destination((lat, lon), bearing)
    return destination.latitude, destination.longitude


lamps['N'] = lamps.apply(lambda row: calculate_destination_coordinates(row['Y'], row['X'], 20, 0)[0], axis=1)
lamps['E'] = lamps.apply(lambda row: calculate_destination_coordinates(row['Y'], row['X'], 20, 90)[1], axis=1)
lamps['S'] = lamps.apply(lambda row: calculate_destination_coordinates(row['Y'], row['X'], 20, 180)[0], axis=1)
lamps['W'] = lamps.apply(lambda row: calculate_destination_coordinates(row['Y'], row['X'], 20, 270)[1], axis=1)


In [ ]:
#Lamps Directional

lampsDir = pd.DataFrame(columns=['startTime','endTime','latitude','longitude','direction'])

import math

def calculate_heading(lat1, lon1, lat2, lon2):
    # Calculate the initial bearing between two points
    delta_lon = lon2 - lon1
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(delta_lon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - \
        math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(delta_lon))
    bearing = math.atan2(x, y)
    
    # Convert bearing to degrees and normalize to [0, 360) range
    heading = (math.degrees(bearing) + 360) % 360
    
    return heading


for l in range (1,len(lamps)):
    
    lamp = lamps.iloc[l]    
    sourceDf = movement.loc[movement.apply(lambda x: (lamp.N>x['latitude'] and lamp.S<x['latitude'] and 
                                                      lamp.E>x['longitude'] and lamp.W<x['longitude']),axis=1)]
    if len(sourceDf)<1:
        continue
    
    mintime = datetime.datetime.strptime(sourceDf.iloc[0].time, "%Y-%m-%d %H:%M:%S")
    prevtime = mintime
    for i in range (1,len(sourceDf)):
        current = datetime.datetime.strptime(sourceDf.iloc[i].time, "%Y-%m-%d %H:%M:%S")
        lampsDir.loc[len(lampsDir.index)] = [current, current + datetime.timedelta(seconds=10), lamp.Y, lamp.X,
                                             calculate_heading(lamp.Y, lamp.X, sourceDf.iloc[i]['latitude'],
                                             sourceDf.iloc[i]['longitude'])]
            
lampsDir.to_csv("lampDir.csv", index=False)
